In [8]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize

In [9]:
path = "D:/BrainDead/nextwordprediction/1661-0.txt"

In [10]:
with open(path , 'r' ,encoding='utf-8' ) as file :
    data = file.read().lower()

In [13]:
sentences = sent_tokenize(data)

In [80]:
tk = Tokenizer(oov_token='<OOV>')

In [14]:
for i in range(len(sentences)):
    arr = []
    for j in sentences[i].strip().split():
        if j[0] != '\ufeff':
            arr.append(j)
    sentences[i] = ' '.join(arr)

In [17]:
complete_text = ' '.join(sentences)

In [19]:
from keras.preprocessing.text import Tokenizer

In [20]:
tokens = Tokenizer(oov_token='<OOV>')

In [21]:
tokens.fit_on_texts([complete_text])

In [28]:
transformed_text = []
for i in sentences:
    n = len(tokens.texts_to_sequences([i])[0])
    for j in range(1,n):
        transformed_text.append(tokens.texts_to_sequences([i])[0][:j+1])

In [33]:
maxi = [len(i) for i in transformed_text ]

In [34]:
max_len = max(maxi)

In [36]:
from keras.utils import pad_sequences

In [38]:
transformed_arr= pad_sequences(transformed_text , max_len , padding='pre')

In [41]:
features = transformed_arr[:,:-1] 
output = transformed_arr[:,-1]

In [43]:
from keras.utils import to_categorical

In [59]:
output_encoded = to_categorical(output , num_classes=len(tokens.word_index)+1)

In [64]:
transformed_arr.shape

(106445, 251)

In [62]:
import tensorflow
from tensorflow import keras
from keras.layers import LSTM , Dense , Embedding
from keras.models import Sequential

In [101]:
model = Sequential()
model.add(Embedding(len(tokens.word_index)+1 , 100 , input_length=len(transformed_arr[0])))
model.add(LSTM(150))
model.add(Dense(len(tokens.word_index)+1 , activation='softmax' ))
model.compile(loss = 'categorical_crossentropy' , metrics = ['accuracy'] )

In [102]:
model.fit(transformed_arr , output_encoded , epochs = 10)

Epoch 1/10
3327/3327 [==============================] - 1220s 365ms/step - loss: 5.5639 - accuracy: 0.2235
Epoch 2/10
3327/3327 [==============================] - 27684s 8s/step - loss: 3.9746 - accuracy: 0.4933
Epoch 3/10
3327/3327 [==============================] - 1138s 342ms/step - loss: 3.5235 - accuracy: 0.5556
Epoch 4/10
3327/3327 [==============================] - 749s 225ms/step - loss: 3.2672 - accuracy: 0.5929
Epoch 5/10
3327/3327 [==============================] - 1164s 350ms/step - loss: 3.0435 - accuracy: 0.6297
Epoch 6/10
3327/3327 [==============================] - 1061s 319ms/step - loss: 2.8547 - accuracy: 0.6584
Epoch 7/10
3327/3327 [==============================] - 753s 226ms/step - loss: 2.7000 - accuracy: 0.6780
Epoch 8/10
3327/3327 [==============================] - 603s 181ms/step - loss: 2.5662 - accuracy: 0.6954
Epoch 9/10
3327/3327 [==============================] - 2040s 613ms/step - loss: 2.4489 - accuracy: 0.7087
Epoch 10/10
3327/3327 [===================

In [131]:
word = 'email'

In [132]:
tokenized = tokens.texts_to_sequences([word])

In [133]:
padded = pad_sequences(tokenized , maxlen= max_len , padding='pre')

In [134]:
predicted = model.predict(padded)

1/1 [==============================] - 0s 45ms/step


In [135]:
max_index = np.argmax(predicted)

In [136]:
tokens.index_word[max_index]

'fashion'

In [137]:
sentences[100]

'i am lost without my boswell.'

In [158]:
word = 'I love '
for i in range(2) :
    tokenized = tokens.texts_to_sequences([word])[0]
    padded = pad_sequences([tokenized] , maxlen= max_len , padding='pre')
    predicted = model.predict(padded)
    max_index = np.argmax(predicted)
    word = word + ' ' + tokens.index_word[max_index]
    print(word)

1/1 [==============================] - 0s 35ms/step
I love  singular
1/1 [==============================] - 0s 36ms/step
I love  singular singular


In [159]:
import pickle

In [161]:
with open('D:/NextWordPredictor' , 'wb') as file:
    pickle.dump(model , file= file)

In [162]:
with open('D:/NextWordPredictortokenizer' , 'wb') as file:
    pickle.dump(tokens , file= file)